In [1]:
"""
Organizing good plots for syntax, fo|cusing on activity before trial onset.
- euclidian dist
- state space

See structured representation?

"""

'\nOrganizing good plots for syntax, fo|cusing on activity before trial onset.\n- euclidian dist\n- state space\n\nSee structured representation?\n\n'

In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from neuralmonkey.scripts.analy_dfallpa_extract import extract_dfallpa_helper
from neuralmonkey.classes.population_mult import load_handsaved_wrapper, dfpa_match_chans_across_pa_each_bregion
from neuralmonkey.classes.population_mult import extract_single_pa
from neuralmonkey.metadat.analy.anova_params import params_getter_euclidian_vars
from neuralmonkey.classes.population_mult import dfpa_concatbregion_preprocess_clean_bad_channels, dfpa_concat_bregion_to_combined_bregion, dfpa_concatbregion_preprocess_wrapper
from pythonlib.tools.pandastools import append_col_with_grp_index
import seaborn as sns
import os
from neuralmonkey.classes.population_mult import extract_single_pa
from neuralmonkey.analyses.state_space_good import euclidian_distance_compute_trajectories_single, euclidian_distance_compute_trajectories


# Load DFallPa dataset

In [ ]:
version = "trial"
combine = False
question = "RULE_BASE_trial"

animal = "Diego"
# date = 230915
# date = 230913
date = 230726

# animal = "Pancho"
# # date = 230829
# date = 220908

# Load a single DFallPA
DFallpa = load_handsaved_wrapper(animal, date, version=version, combine_areas=combine, 
                                    question=question)

events_keep = ["03_samp"]
DFallpa = DFallpa[DFallpa["event"].isin(events_keep)].reset_index(drop=True)

DFallpa = dfpa_concat_bregion_to_combined_bregion(DFallpa)


# Make a copy of all PA before normalization
dfpa_concatbregion_preprocess_wrapper(DFallpa, animal, date)

### Preprocessing
from neuralmonkey.scripts.analy_syntax_good_eucl_trial import preprocess_dfallpa_basic_quick
preprocess_dfallpa_basic_quick(DFallpa)

### Subspace projection

In [ ]:
SAVEDIR = "/tmp"


In [ ]:
DFallpa = DFallpa[:15].reset_index(drop=True)
DFallpa

In [ ]:
from neuralmonkey.scripts.analy_syntax_good_eucl_trial import preprocess_dfallpa
list_subspace_projection = ["epch_sytxcncr"]
tbin_dur = "default"
tbin_slide = None

LIST_VAR_VAROTHERS = [
    # ("syntax_slot_0", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "seqc_0_shape", "syntax_slot_1"]),
    # ("syntax_slot_0", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "syntax_slot_1"]),
    # ("syntax_slot_0", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "seqc_0_shape"]),
    # ("syntax_slot_0", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc"]),
    # ("syntax_slot_0", ["FEAT_num_strokes_beh", "epoch"]),
    # ("syntax_slot_1", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "syntax_slot_0"]),
    # ("syntax_concrete", ["FEAT_num_strokes_beh", "epoch", "seqc_0_shape", "seqc_0_loc"]),
    ("syntax_concrete", ["FEAT_num_strokes_beh", "epoch"]),
    # ("FEAT_num_strokes_beh", ["epoch", "seqc_0_loc", "seqc_0_shape"]),
    # ("epoch", ["FEAT_num_strokes_beh", "seqc_0_loc", "seqc_0_shape"]),
]

for subspace_projection in list_subspace_projection:

    SAVEDIR_ANALYSIS = f"{SAVEDIR}/statespace_and_regression/{animal}-{date}-comb={combine}-q={question}-ssproj={subspace_projection}"            
    os.makedirs(SAVEDIR_ANALYSIS, exist_ok=True)

    ### Preprocessing
    LIST_DIMS = preprocess_dfallpa(DFallpa, subspace_projection, tbin_slide, tbin_dur, SAVEDIR_ANALYSIS, HACK=False)



### State space plots

In [ ]:
LIST_VAR_VAROTHERS = [
    ("syntax_slot_0", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "seqc_0_shape", "syntax_slot_1"]),
    ("syntax_slot_0", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "syntax_slot_1"]),
    ("syntax_slot_0", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "seqc_0_shape"]),
    ("syntax_slot_0", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc"]),
    ("syntax_slot_0", ["FEAT_num_strokes_beh", "epoch"]),
    ("syntax_slot_1", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "syntax_slot_0"]),
    ("syntax_concrete", ["FEAT_num_strokes_beh", "epoch", "seqc_0_shape", "seqc_0_loc"]),
    ("syntax_concrete", ["FEAT_num_strokes_beh", "epoch"]),
]
NPCS_KEEP = 8

In [ ]:
from neuralmonkey.scripts.analy_syntax_good_eucl_trial import preprocess_dfallpa

list_subspace_projection = ["epch_sytxcncr"]
tbin_dur = "default"
tbin_slide = None

for subspace_projection in list_subspace_projection:

    ### Preprocessing
    savedir = f"{SAVEDIR}/ssproj={subspace_projection}"
    os.makedirs(savedir, exist_ok=True)
    LIST_DIMS = preprocess_dfallpa(DFallpa, subspace_projection, tbin_slide, tbin_dur, savedir, HACK=False)


In [ ]:
from neuralmonkey.scripts.analy_syntax_good_eucl_trial import state_space_plot_scalar_wrapper

# extract new variable, the number of items for each of the chunk ranks
SAVEDIR = f"/tmp/SYNTAX_TRIAL/{animal}-{date}"
os.makedirs(SAVEDIR, exist_ok=True)
state_space_plot_scalar_wrapper(DFallpa, SAVEDIR, LIST_VAR_VAROTHERS)

### Linear regression to find encoding axis

In [ ]:
SAVEDIR_BASE = f"/tmp/SYNTAX_REGRE/{animal}-{date}"


In [ ]:
from neuralmonkey.scripts.analy_syntax_good_eucl_trial import regression_fit_and_score_single, regression_wrapper
var_effect = "syntax_slot_0"
vars_others = ["FEAT_num_strokes_beh", "epoch", "seqc_0_shape", "seqc_0_loc", "syntax_slot_1"]
# vars_others = ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "syntax_slot_1"]
# vars_others = ["FEAT_num_strokes_beh", "epoch", "syntax_slot_1"]

DFRES_ALL, DFDISTCOS_ALL, SAVEDIR = regression_wrapper(DFallpa, var_effect, vars_others, SAVEDIR_BASE, 
                                              ndims = 4, PLOT_SCATTER_PRED_VS_ACTUAL = False)


In [ ]:
### For pruning
from neuralmonkey.scripts.analy_syntax_good_eucl_trial import regression_wrapper_prune

min_levs = 3
# min_r2_test = 0.1
min_r2_test = None
min_ntot = 15
min_nmin = 4

DFRES_ALL_PRUNED, DFDISTCOS_ALL_PRUNED = regression_wrapper_prune(DFRES_ALL, DFDISTCOS_ALL, min_levs, min_r2_test, 
                                                                  min_ntot, min_nmin)

In [ ]:
# for each bregion/event, 
# Plot:
# - distribution of R2
# - how many R2 are significant
# - cosine between R2
# --- both with and without significant

In [ ]:
savedir = f"{SAVEDIR}/MULT_PLOTS"
os.makedirs(savedir, exist_ok=True)

In [ ]:
from neuralmonkey.scripts.analy_syntax_good_eucl_trial import regression_wrapper_plot_each_grp, regression_wrapper_plot_cosine_sim


# Prune, based on data quality (not on r2)
min_levs = 3
min_r2_test = None
min_ntot = 20
min_nmin = 5

DFRES_ALL_PRUNED, DFDISTCOS_ALL_PRUNED = regression_wrapper_prune(DFRES_ALL, DFDISTCOS_ALL, min_levs, min_r2_test, 
                                                                  min_ntot, min_nmin)

# dfthis = DFRES_ALL[DFRES_ALL["same_grp"] == True].reset_index(drop=True)
for savesuff, dfthis in zip(
    ["all", "pruned"],
    [DFRES_ALL, DFRES_ALL_PRUNED]):

    savedirthis = f"{savedir}/plot_each_grp-{savesuff}"
    os.makedirs(savedirthis, exist_ok=True)
    regression_wrapper_plot_each_grp(dfthis, savedirthis)

    plt.close("all")

In [ ]:
# Prune, based on good regression
min_levs = 3
min_r2_test = 0.1
min_ntot = 20
min_nmin = 5

DFRES_ALL_PRUNED, DFDISTCOS_ALL_PRUNED = regression_wrapper_prune(DFRES_ALL, DFDISTCOS_ALL, min_levs, min_r2_test, 
                                                                  min_ntot, min_nmin)

for savesuff, dfthis in zip(
    ["all", "pruned"],
    [DFDISTCOS_ALL, DFDISTCOS_ALL_PRUNED]):

    savedirthis = f"{savedir}/plot_cosine_sim-{savesuff}"
    os.makedirs(savedirthis, exist_ok=True)

    # dfthis = DFDISTCOS_ALL[DFDISTCOS_ALL["same_grp"] == False].reset_index(drop=True)
    regression_wrapper_plot_cosine_sim(dfthis, savedir)

In [ ]:

date

In [ ]:

##### Plot state space scatter
from neuralmonkey.analyses.state_space_good import trajgood_plot_colorby_splotby_scalar_WRAPPER
savedir_this = "/tmp"
LIST_DIMS = [(0,1), (2,3), (4,5), (5,6)]
dflab = PAredu.Xlabels["trials"]
trajgood_plot_colorby_splotby_scalar_WRAPPER(X, dflab, var_effect, savedir_this,
                                                vars_subplot=vars_others, list_dims=LIST_DIMS,
                                                overlay_mean_orig=True)


In [ ]:
### IN PROGRESS

In [ ]:
### Project neural data onto the axis.

# - get predicted neural for min and max y
from pythonlib.tools.vectools import projection_onto_axis_subspace

grp_axis = (5, "llCV2", "line-6-2-0", (2, 0), 2)
grp_data = (5, "llCV2", "line-6-2-0", (0, 0), 2)

_, _, _, _, reg, _, _ = _extract_data(X, Y, grp_axis, grp_axis)
X_train, labels_train, _, _, _, _, _ = _extract_data(X, Y, grp_data, grp_data)

reg.coef_

x1 = -reg.coef_
x2 = reg.coef_

projection_onto_axis_subspace(x1, x2, X_train, True, plot_color_labels=labels_train)
projection_onto_axis_subspace(x1, x2, X_train, True)

### Score euclidian distance

In [ ]:
# TODO

### [Good] project to submspace spanned by two axes, each identified using regression (targeted PCA)

In [ ]:
# TODO:
- read papers that do this.

In [ ]:
# Ways to get factoriztaion:
# 1. average over other variables (problem if they are correlated)
# 2. condition on other variables (problem if limited data)
# 3. first use multiple (or partial) regression model 
# -- Liping 

# Ways to identify subspace
# 1. PCA, after factorization.
# 2. Regression (without necessarily factorizing).

# TO read:
# -  machens.
# - churchland 2012


# TODO:
# Like in Mante 2013
# - get denoising matrix.
# - compute regression 
# 

In [ ]:
### For each neuron/dim, get regression coefficients (task coeff)
# Method 1: Full data, multple regression

# Method 2: 

In [ ]:
### method: regression

# variables = ["seqc_0_loc", "seqc_0_shape", "syntax_slot_1", "syntax_slot_2"]
# variables_is_cat = [True, True, False, False]

# variables = ["seqc_0_shape"]
# variables_is_cat = [True]



In [ ]:
### Entire pipeline

In [ ]:
# PA = DFallpa["pa"].values[7]


In [ ]:
from pythonlib.tools.pandastools import extract_with_levels_of_conjunction_vars_helper

extract_with_levels_of_conjunction_vars_helper(dflab, var=var_effect, vars_others=vars_others, 
                                               n_min_per_lev=prune_levs_min_n_trials, lenient_allow_data_if_has_n_levels=3)

In [ ]:
pa = DFallpa["pa"].values[0]
dflab = pa.Xlabels["trials"]

from pythonlib.tools.pandastools import extract_with_levels_of_conjunction_vars_helper


In [ ]:
dflab["syntax_concrete"].value_counts()

In [ ]:
dflab["syntax_slot_2"].value_counts()

In [ ]:
# Print levels for the varialbes
from pythonlib.tools.pandastools import grouping_print_n_samples
dflab = PA.Xlabels["trials"]
grouping_print_n_samples(dflab, variables)


In [ ]:
dflab = PAredu.Xlabels["trials"]
dfout, _ = extract_with_levels_of_conjunction_vars_helper(dflab, var=var_effect, vars_others=["syntax_slot_1"], 
                                        n_min_per_lev=3, 
                                        lenient_allow_data_if_has_n_levels=2)        
len(dfout)

In [ ]:
var_effect, vars_others

In [ ]:
pa, vars_others = PAredu.slice_prune_dflab_and_vars_others(var_effect, vars_others,
                                                prune_levs_min_n_trials, min_levs_per_levother)
vars_others

In [ ]:
# During samp
DFallpa = DFallpa[:16]

from pythonlib.tools.vectools import average_vectors_wrapper, get_vector_from_angle

twind_scal = (0.2, 1.4)
from neuralmonkey.scripts.analy_syntax_good_eucl_trial import state_space_targeted_pca_scalar_single, targeted_pca_euclidian_dist_angles

variables = ["epoch", "FEAT_num_strokes_beh", "seqc_0_loc", "seqc_0_shape", "syntax_slot_0", "syntax_slot_1", "syntax_slot_2"]
variables_is_cat = [True, True, True, True, False, False, False]

list_subspaces = [
    ("syntax_slot_0", "syntax_slot_1"),
    # ("syntax_slot_1", "syntax_slot_2"),   
    # ("syntax_slot_0", "syntax_slot_2"),   
]

LIST_VAR_VAROTHERS_SS = [
    # ("syntax_slot_ratio", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "seqc_0_shape"]),
    # ("syntax_slot_ratio", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc"]),
    # ("syntax_slot_ratio", ["FEAT_num_strokes_beh", "epoch"]),

    # ("shapes_n_unique", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "seqc_0_shape"]),

    ("syntax_slot_0", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "seqc_0_shape", "syntax_slot_1"]),
    # ("syntax_slot_0", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "syntax_slot_1"]),
    # ("syntax_slot_0", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "seqc_0_shape"]),
    # ("syntax_slot_0", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc"]),
    ("syntax_slot_0", ["FEAT_num_strokes_beh", "epoch"]),

    ("syntax_slot_1", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "seqc_0_shape", "syntax_slot_0"]),
    # ("syntax_slot_1", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "syntax_slot_0"]),
    ("syntax_slot_1", ["FEAT_num_strokes_beh", "epoch"]),

    # ("syntax_slot_2", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "seqc_0_shape", "syntax_slot_0", "syntax_slot_1"]),
    # ("syntax_slot_2", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "seqc_0_shape", "syntax_slot_0"]),
    # ("syntax_slot_2", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "seqc_0_shape", "syntax_slot_1"]),
    # ("syntax_slot_2", ["FEAT_num_strokes_beh", "epoch"]),

    ("syntax_concrete", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "seqc_0_shape"]),
    ("syntax_concrete", ["FEAT_num_strokes_beh", "epoch"]),

    # ("FEAT_num_strokes_beh", ["epoch", "seqc_0_loc", "seqc_0_shape"]),
    # ("epoch", ["FEAT_num_strokes_beh", "seqc_0_loc", "seqc_0_shape"]),
    ("seqc_0_shape", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc"]),
]

LIST_DIMS = [(0,1), (1,2)]

LIST_VAR_VAROTHERS_REGR = [
    ("syntax_slot_0", ["syntax_slot_1", "seqc_0_loc", "seqc_0_shape", "epoch", "FEAT_num_strokes_beh"]),
    ("syntax_slot_1", ["syntax_slot_0", "seqc_0_loc", "seqc_0_shape", "epoch", "FEAT_num_strokes_beh"]),
    # ("FEAT_num_strokes_beh", ["epoch", "seqc_0_loc", "seqc_0_shape"]),
]
# else:
#     # TODO: find the most constrainig params givne this expt
#     LIST_VAR_VAROTHERS_REGR = [
#         ("syntax_slot_0", ["epoch", "seqc_0_loc", "seqc_0_shape", "syntax_slot_1"]),
#         ("syntax_slot_1", ["epoch", "seqc_0_loc", "seqc_0_shape", "syntax_slot_0"]),
#         # ("FEAT_num_strokes_beh", ["epoch", "seqc_0_loc", "seqc_0_shape"]),
#     ]


subspace_tuple = ('syntax_slot_0', 'syntax_slot_1')

SAVEDIR_ANALYSIS = f"/tmp/TERGET_PCA_EUCL/{animal}-{date}"
os.makedirs(SAVEDIR_ANALYSIS, exist_ok=True)
DFANGLE = targeted_pca_euclidian_dist_angles(DFallpa, SAVEDIR_ANALYSIS, twind_scal,
                                       variables, variables_is_cat, list_subspaces, LIST_VAR_VAROTHERS_SS, # For dim reduction and plotting state space
                                       subspace_tuple, LIST_VAR_VAROTHERS_REGR)


In [ ]:
from neuralmonkey.scripts.analy_syntax_good_eucl_trial import targeted_pca_euclidian_dist_angles_plots

# var_vector_length = "dist_norm"
var_vector_length = "dist_yue_diff"
# length_method = "dot"
length_method = "sum"
min_levs_exist = 3
SAVEDIR = f"{SAVEDIR_ANALYSIS}/PLOTS/varlength={var_vector_length}-lengthmeth={length_method}-minlevs={min_levs_exist}"
os.makedirs(SAVEDIR, exist_ok=True)

targeted_pca_euclidian_dist_angles_plots(DFANGLE, var_vector_length, length_method, min_levs_exist, SAVEDIR)

##### Other methods for doing targeted PCA

In [ ]:
dim_red_method = "pca"

# Compute PCA projection matrix
scalar_or_traj = "scal"
savedir = "/tmp"
twind_pca = twind_scal
tbin_dur = "default"
tbin_slide = "default"
NPCS_KEEP = 20

dpca_var = "var_all_conditions"
dpca_vars_group = None

_, _, pca = PA.dataextract_dimred_wrapper(scalar_or_traj, dim_red_method, savedir, 
                                   twind_pca, tbin_dur, tbin_slide, 
                                   NPCS_KEEP, dpca_var, dpca_vars_group = dpca_vars_group, dpca_filtdict=None, dpca_proj_twind = None, 
                                   raw_subtract_mean_each_timepoint=False,
                                   n_min_per_lev_lev_others = 2, return_pca_components=True)

print(pca["components"].shape) # (ndim_reduced, ndim_orig)

In [ ]:
# De-noise the coefficients using pca

# Project 
p = pca["components"] # (ndims_pc, nchans)

ndims_keep = 20
p = p[:ndims_keep, :]

D = p.T @ p # (nchans, nchans)
values_denoised = D @ dfcoeff.values # (nchans, nfeatures

dfcoeff = pd.DataFrame(values_denoised, columns=dfcoeff.columns)

# Replot coefficiens
from pythonlib.tools.snstools import heatmap
heatmap(dfcoeff, annotate_heatmap=False, diverge=True);

In [ ]:
# [optional] do regression by controlling for specific values of the other variables
from pythonlib.tools.pandastools import extract_with_levels_of_conjunction_vars_helper

# variables = ["epoch", "FEAT_num_strokes_beh", "seqc_0_loc", "seqc_0_shape", "syntax_slot_0", "syntax_slot_1"]
# variables = ["seqc_0_loc", "seqc_0_shape"]

list_var_varothers = [
    ["syntax_slot_0", ("epoch", "FEAT_num_strokes_beh", "seqc_0_loc", "seqc_0_shape", "syntax_slot_1")],
    ["syntax_slot_1", ("epoch", "FEAT_num_strokes_beh", "seqc_0_loc", "seqc_0_shape", "syntax_slot_0")],
]
list_vars_is_cat = [False, False]

n_min_per_lev = 4
dflab = pa.Xlabels["trials"]

print("TODO: Test that all variables are cat or ordinal")
map_vareffect_to_dfcoeff = {}
# for i in range(len(variables)):
#     var_effect = variables[i]
#     vars_others = variables[:i] + variables[i+1:]
#     var_effect_is_cat = variables_is_cat[i]
#     dfout, dict_dfthis = extract_with_levels_of_conjunction_vars_helper(dflab, var_effect, vars_others, n_min_per_lev, lenient_allow_data_if_has_n_levels=2)

for (var_effect, vars_others), var_effect_is_cat in zip(list_var_varothers, list_vars_is_cat):
    dfout, dict_dfthis = extract_with_levels_of_conjunction_vars_helper(dflab, var_effect, vars_others, 
                                                                        n_min_per_lev, lenient_allow_data_if_has_n_levels=2)

    print(var_effect)
    print(var_effect_is_cat)
    print(vars_others)
    print(len(dfout))

    if len(dfout)>0:
        # for each level of var_other, do regression
        # map_varlevel_to_dfcoeff = {}
        list_dfcoeff = []
        PLOT_COEFF_HEATMAP = False
        columns_take = None
        for grp, dfthis in dict_dfthis.items():
            print(grp, len(dfthis))

            # Sanity check indices are correct, even globally.
            inds = dfthis["_index"]
            dflab.iloc[inds]["trialcode"].tolist() == dfthis["trialcode"].tolist()
        
            pasub = pa.slice_by_dim_indices_wrapper("trials", inds, reset_trial_indices=True)
            dfcoeff, res_all = pasub.regress_neuron_task_variables_all_chans([var_effect], [var_effect_is_cat], 
                                                                            PLOT_COEFF_HEATMAP=PLOT_COEFF_HEATMAP, PRINT=False)
            
            # save the coefficients
            _columns_take = [k for k,v in res_all[0]["original_feature_mapping"].items() if v==var_effect]
            if columns_take is not None:
                assert columns_take == _columns_take, "to ensure that concatting at end is correct"
            else:
                columns_take = _columns_take

            if len(columns_take)>1:
                print(columns_take)
                print(var_effect)
                print("var_effect_is_cat:", var_effect_is_cat)
                assert False, "not yet coded for categorical variables with >2 categories. TO do so, need to solve problem of diff var_other levesl having ptoentialyl different levesl of var_effect, and how to agg that over var_others"

            dfcoeff_nointercept = dfcoeff.loc[:, columns_take]

            list_dfcoeff.append(dfcoeff_nointercept)
            
        # Get mean coefficients over all runs
        dfcoeff_mean = np.concatenate(list_dfcoeff, axis=1).mean(axis=1, keepdims=True) # (ndims, ncoeff)

        if PLOT_COEFF_HEATMAP:
            print(columns_take)
            print(dfcoeff_mean.shape)
            assert False
        map_vareffect_to_dfcoeff[var_effect] = dfcoeff_mean
    
dfcoeff_all_var = np.concatenate(list(map_vareffect_to_dfcoeff.values()), axis=1)
dfcoeff_all_var = pd.DataFrame(dfcoeff_all_var, columns=[x[0] for x in list_var_varothers])

# Hacky, for plotting
dfcoeff = dfcoeff_all_var

### Compare to my previous DPCA method

In [ ]:
scalar_or_traj = "scal"
dim_red_method = "dpca"
savedir = "/tmp"
twind_pca = twind_scal
tbin_dur = "default"
tbin_slide = "default"
NPCS_KEEP = 20

# dpca_var = "seqc_0_shape"
# dpca_vars_group = None


list_var_varothers = [
    ["syntax_slot_0", ("epoch", "FEAT_num_strokes_beh", "seqc_0_loc", "seqc_0_shape", "syntax_slot_1")],
    ["syntax_slot_1", ("epoch", "FEAT_num_strokes_beh", "seqc_0_loc", "seqc_0_shape", "syntax_slot_0")],
]
list_vars_is_cat = [False, False]

# dpca_var = "syntax_slot_0"
# dpca_vars_group = ["epoch", "FEAT_num_strokes_beh", "seqc_0_loc", "seqc_0_shape", "syntax_slot_1"]
dpca_var = "syntax_slot_1"
dpca_vars_group = ["epoch", "FEAT_num_strokes_beh", "seqc_0_loc", "seqc_0_shape", "syntax_slot_0"]

Xredu, PAredu, pca = PA.dataextract_dimred_wrapper(scalar_or_traj, dim_red_method, savedir, 
                                   twind_pca, tbin_dur, tbin_slide, 
                                   NPCS_KEEP, dpca_var, dpca_vars_group = dpca_vars_group, dpca_filtdict=None, dpca_proj_twind = None, 
                                   raw_subtract_mean_each_timepoint=False,
                                   umap_n_components=2, umap_n_neighbors=40,
                                   inds_pa_fit=None, inds_pa_final=None,
                                   n_min_per_lev_lev_others = 2, return_pca_components=True)

In [ ]:
pca["X_before_dimred"].shape

In [ ]:
pca["components"].shape # (ndims, ntrial)

In [ ]:
# plot state space
savedir = "/tmp/TARGETED_DPCA/original"
os.makedirs(savedir, exist_ok=True)
LIST_VAR_VAROTHERS_SS = [
    # ("syntax_slot_ratio", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "seqc_0_shape"]),
    # ("syntax_slot_ratio", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc"]),
    # ("syntax_slot_ratio", ["FEAT_num_strokes_beh", "epoch"]),

    # ("shapes_n_unique", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "seqc_0_shape"]),

    ("syntax_slot_0", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "seqc_0_shape", "syntax_slot_1"]),
    ("syntax_slot_0", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "syntax_slot_1"]),
    ("syntax_slot_0", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "seqc_0_shape"]),
    ("syntax_slot_0", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc"]),
    ("syntax_slot_0", ["FEAT_num_strokes_beh", "epoch"]),

    ("syntax_slot_1", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "seqc_0_shape", "syntax_slot_0"]),
    ("syntax_slot_1", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "syntax_slot_0"]),

    # ("syntax_concrete", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "seqc_0_shape"]),
    # ("syntax_concrete", ["FEAT_num_strokes_beh", "epoch"]),

    # ("FEAT_num_strokes_beh", ["epoch", "seqc_0_loc", "seqc_0_shape"]),
    # ("epoch", ["FEAT_num_strokes_beh", "seqc_0_loc", "seqc_0_shape"]),
    # ("seqc_0_shape", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc"]),
    # ("seqc_0_loc", ["FEAT_num_strokes_beh", "epoch", "seqc_0_shape"]),
]

LIST_VAR = [x[0] for x in LIST_VAR_VAROTHERS_SS]
LIST_VARS_OTHERS = [x[1] for x in LIST_VAR_VAROTHERS_SS]
PAredu.plot_state_space_good_wrapper(savedir, LIST_VAR, LIST_VARS_OTHERS)

In [ ]:
# TODO:
# - instead of multiple regression, do controlled regression
# - do regression in neural space
# - do dPCA, controlling for each variable, and then concatenate the top PC
# - do PCA after regression


In [ ]:
# FINAL SCORE

In [ ]:
# 

##### Debug

In [ ]:
### Plot showing that the order doesnt matter, first 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression

# Set random seed for reproducibility
np.random.seed(42)

# Parameters
n_neurons = 100
n_trials = 500
n_signal_neurons = 5
noise_level = 10.0  # High noise
signal_strength = 1.0
k = 2

# Simulate task variable (binary)
task = np.random.choice([0, 1], size=n_trials)

# Simulate neural data
X = np.random.randn(n_neurons, n_trials) * noise_level  # Pure noise

# Add task-related signal to a few neurons
for i in range(n_signal_neurons):
    X[i, :] += task * signal_strength

# Center data
X_centered = X - X.mean(axis=1, keepdims=True)

# --- Option A: Regress first, then denoise ---

# Step 1: Regress each neuron on task
beta = np.zeros(n_neurons)
for i in range(n_neurons):
    model = LinearRegression().fit(task.reshape(-1, 1), X_centered[i, :])
    beta[i] = model.coef_[0]

# Step 2: PCA denoising (keep top k PCs)
pca = PCA(n_components=k)
pca.fit(X_centered.T)
P = pca.components_.T @ pca.components_
beta_denoised = P @ beta

# --- Option B: Denoise first, then regress ---

# Step 1: Denoise data
X_denoised = P @ X_centered

# Step 2: Regress each neuron in denoised data on task
beta_denoised_first = np.zeros(n_neurons)
for i in range(n_neurons):
    model = LinearRegression().fit(task.reshape(-1, 1), X_denoised[i, :])
    beta_denoised_first[i] = model.coef_[0]

# Plot results
fig, axs = plt.subplots(1, 3, figsize=(15, 5))

# True task neurons
axs[0].scatter(range(n_neurons), beta, color='gray')
axs[0].scatter(range(n_signal_neurons), beta[:n_signal_neurons], color='red')
axs[0].set_title('Original Beta (regress first)')
axs[0].axhline(0, color='black', linestyle='--')

# After denoising betas
axs[1].scatter(range(n_neurons), beta_denoised, color='gray')
axs[1].scatter(range(n_signal_neurons), beta_denoised[:n_signal_neurons], color='red')
axs[1].set_title('Beta after denoising (regress first)')
axs[1].axhline(0, color='black', linestyle='--')

# After denoising data first
axs[2].scatter(range(n_neurons), beta_denoised_first, color='gray')
axs[2].scatter(range(n_signal_neurons), beta_denoised_first[:n_signal_neurons], color='red')
axs[2].set_title('Beta after denoising data first')
axs[2].axhline(0, color='black', linestyle='--')

plt.tight_layout()

plt.show()


### [Load mult dates] [Old method, angles]

In [ ]:
# NOTE: This is obsolete, but can run this to plot, and it works. For updated version
# see below (### [MULT] [NEW] Load and plot all data)

In [ ]:
from pythonlib.tools.pandastools import grouping_append_and_return_inner_items_good
from pythonlib.tools.vectools import get_vector_from_angle


In [ ]:
# # list_dates = [230728, 231118, 240822, 230723, 230724, 230726, 230727, 230730, 230815, 230816, 230817, 230913, 230914, 230915, 231116, 240827, 250319, 250321]
# list_dates = [230728, 231118, 240822, 230723, 230724, 230726, 230727, 230815, 230816, 230817, 230913, 230914, 230915, 231116, 250321]
# list_dates = [230728, 230723, 230724, 230726, 230727, 230815, 230816, 230817, 230913, 230914, 230915]
# animal = "Diego"

# list_dates = [230810, 230811, 230824, 230826, 230829, 231114, 231116, 240830, 220831, 220901, 250321, 250322, 220902, 220906, 220907, 220908, 220909]
# list_dates = [230810, 230811, 230824, 230826, 230829, 231114, 231116, 240830, 220901, 250321, 250322, 220902, 220906, 220907, 220908, 220909]
list_dates = [220901, 250321, 220902, 220906, 220907, 220908, 220909]
animal = "Pancho"

SAVEDIR = "/lemur2/lucas/analyses/recordings/main/syntax_good_trial/targeted_dim_redu_angles"
# Diego-230723-comb=False-q=RULE_BASE_trial

var_vector_length = "dist_yue_diff"
# var_vector_length = "dist_norm"
# length_method = "sum"
length_method = "dot"
min_levs_exist = 3


list_dfangle = []
list_dfdot = []
for date in list_dates:
    path = f"{SAVEDIR}/{animal}-{date}-comb=False-q=RULE_BASE_trial/DFANGLE.pkl"
        
        # import pickle
        # with open(path, "rb") as f:
        
    dfangle = pd.read_pickle(path)
    dfangle["animal"] = animal
    dfangle["date"] = date
    dfangle["levs_exist_n"] = [len(levs_exist) for levs_exist in dfangle["levs_exist"]]

    dfangle = dfangle[
        (dfangle["var_vector_length"] == var_vector_length) & 
        (dfangle["length_method"] == length_method) &
        (dfangle["levs_exist_n"] >= min_levs_exist)].reset_index(drop=True)

    if len(dfangle)==0:
        print("skipping, dfangle 0: ", dfangle)
        continue

    list_dfangle.append(dfangle)

    ##############
    from neuralmonkey.scripts.analy_syntax_good_eucl_trial import targeted_pca_euclidian_dist_angles_compute_dfdot
    df_dot = targeted_pca_euclidian_dist_angles_compute_dfdot(dfangle, var_vector_length, 
                                                            length_method, min_levs_exist)
    if df_dot is not None:
        df_dot["animal"] = animal
        df_dot["date"] = date
        list_dfdot.append(df_dot)
    
    # assert np.all(_dfdot["dot_product"] == df_dot["dot_product"])
    # # assert np.all(_dfdot["var_effect_12"] == df_dot["var_effect_12"])
    # assert np.all(_dfdot["var_effect_1"] == df_dot["var_effect_1"])
    # assert np.all(_dfdot["var_effect_2"] == df_dot["var_effect_2"])
    # assert np.all(_dfdot["var_other_1"] == df_dot["var_other_1"])
    # assert np.all(_dfdot["var_other_2"] == df_dot["var_other_2"])


    # dfangle["vector_mean"] = [l * get_vector_from_angle(a) for a, l in zip(dfangle["angle_mean"], dfangle["norm_mean"])]

    # res_dot = []
    # grpdict = grouping_append_and_return_inner_items_good(dfangle, ["bregion", "event", "var_effect", "var_other"])
    # for i, (grp1, inds1) in enumerate(grpdict.items()):
    #     for j, (grp2, inds2) in enumerate(grpdict.items()):
    #         if grp1[0]==grp2[0]: # same bregion
    #             bregion =grp1[0]
    #             if grp1[1]==grp2[1]: # same event
    #                 event = grp1[1]
    #                 if j>i:
    #                     tmp1 = dfangle.iloc[inds1]
    #                     tmp2 = dfangle.iloc[inds2]
    #                     assert len(tmp1)==1
    #                     assert len(tmp2)==1

    #                     vec1 = tmp1["vector_mean"].values[0]
    #                     vec2 = tmp2["vector_mean"].values[0]
                        
    #                     # Get dot product
    #                     res_dot.append({
    #                         # "dot_product_mean":np.mean(dot_products),
    #                         "var_effect_1":grp1[2],
    #                         "var_effect_2":grp2[2],
    #                         "var_other_1":grp1[3],
    #                         "var_other_2":grp2[3],
    #                         "dot_product":np.dot(vec1, vec2),
    #                         "bregion":bregion,
    #                         "event":event,
    #                     })
    # if len(res_dot)>0:
    #     df_dot = pd.DataFrame(res_dot)
        # df_dot = append_col_with_grp_index(df_dot, ["var_effect_1", "var_effect_2"], "var_effect_12")
        # df_dot["animal"] = animal
        # df_dot["date"] = date
        # list_dfdot.append(df_dot)


In [ ]:
from pythonlib.tools.pandastools import aggregGeneral
DFANGLE= pd.concat(list_dfangle).reset_index(drop=True)
DF_DOT = pd.concat(list_dfdot).reset_index(drop=True)
print(len(DFANGLE))
print(len(DF_DOT))

In [ ]:
DFANGLE_AGG = aggregGeneral(DFANGLE, ["var_effect", "vars_others", "bregion", "event", "animal", "date"], ["angle_mean", "norm_mean"])
DF_DOT = aggregGeneral(DF_DOT, ["var_effect_1", "var_effect_2", "bregion", "event", "var_effect_12", "animal", "date"], ["dot_product"])

In [ ]:
#"" from neuralmonkey.scripts.analy_syntax_good_eucl_trial import targeted_pca_euclidian_dist_angles_plots
# SAVEDIR_ANALYSIS = "/tmp/SYNTAX_TRIAL_MULT"
# for var_vector_length in ["dist_yue_diff", "dist_norm"]:
#     for length_method in ["sum", "dot"]:
#         for min_levs_exist in [3, 2]:

#             savedir = f"{SAVEDIR_ANALYSIS}/PLOTS/varlength={var_vector_length}-lengthmeth={length_method}-minlevs={min_levs_exist}"
#             os.makedirs(savedir, exist_ok=True)
#             targeted_pca_euclidian_dist_angles_plots(DFANGLE_ALL, var_vector_length, length_method, min_levs_exist, savedir)


In [ ]:
from neuralmonkey.classes.session import MAP_REGION_TO_COMBINED_REGION
MAP_REGION_TO_COMBINED_REGION
DF_DOT["bregion"] = [MAP_REGION_TO_COMBINED_REGION[r] for r in DF_DOT["bregion"]]

In [ ]:
from pythonlib.tools.snstools import rotateLabel

fig = sns.catplot(data=DFANGLE_AGG, x="bregion", y="norm_mean", hue="var_effect", jitter=True, alpha=0.7, aspect=1.5)
rotateLabel(fig)
for ax in fig.axes.flatten():
    ax.axhline(0, color="k", alpha=0.5)
# savefig(fig, f"{SAVEDIR}/allvectors-catplot-1.pdf")

fig = sns.catplot(data=DFANGLE_AGG, x="bregion", y="norm_mean", hue="var_effect", kind="point", aspect=1.5)
rotateLabel(fig)
# savefig(fig, f"{SAVEDIR}/allvectors-catplot-2.pdf")
# plt.close("all")


fig = sns.catplot(data=DFANGLE_AGG, x="bregion", y="norm_mean", hue="var_effect", jitter=True, alpha=0.7, aspect=1.5, col="date", col_wrap=4)
rotateLabel(fig)
for ax in fig.axes.flatten():
    ax.axhline(0, color="k", alpha=0.5)
# savefig(fig, f"{SAVEDIR}/allvectors-catplot-1.pdf")

fig = sns.catplot(data=DFANGLE_AGG, x="bregion", y="norm_mean", hue="var_effect", kind="bar", aspect=1.5, col="date", col_wrap=4)
rotateLabel(fig)
# savefig(fig, f"{SAVEDIR}/allvectors-catplot-2.pdf")
# plt.close("all")


In [ ]:
fig = sns.catplot(data=DF_DOT, x="bregion", y="dot_product", hue="var_effect_12", jitter=True, 
                  alpha=0.25, aspect=1.5)
rotateLabel(fig)
for ax in fig.axes.flatten():
    ax.axhline(0, color="k", alpha=0.5)
# savefig(fig, f"{SAVEDIR}/dot_across_ovar-catplot-1.pdf")

fig = sns.catplot(data=DF_DOT, x="bregion", y="dot_product", hue="var_effect_12", kind="bar", aspect=1.5, errorbar="se")
rotateLabel(fig)
for ax in fig.axes.flatten():
    ax.axhline(0, color="k", alpha=0.5)
# savefig(fig, f"{SAVEDIR}/dot_across_ovar-catplot-2.pdf")

fig = sns.catplot(data=DF_DOT, x="bregion", y="dot_product", hue="var_effect_12", kind="point", aspect=1.5, errorbar="se")
rotateLabel(fig)
for ax in fig.axes.flatten():
    ax.axhline(0, color="k", alpha=0.5)
# savefig(fig, f"{SAVEDIR}/dot_across_ovar-catplot-2.pdf")

fig = sns.catplot(data=DF_DOT, x="bregion", y="dot_product", hue="var_effect_12", kind="violin", aspect=1.5)
rotateLabel(fig)
for ax in fig.axes.flatten():
    ax.axhline(0, color="k", alpha=0.5)
# savefig(fig, f"{SAVEDIR}/dot_across_ovar-catplot-2.pdf")


In [ ]:
fig = sns.catplot(data=DF_DOT, x="bregion", y="dot_product", hue="var_effect_12", jitter=True, 
                  alpha=0.25, aspect=1.5, col="date", col_wrap=4, sharey=False)
rotateLabel(fig)
for ax in fig.axes.flatten():
    ax.axhline(0, color="k", alpha=0.5)
# savefig(fig, f"{SAVEDIR}/dot_across_ovar-catplot-1.pdf")

fig = sns.catplot(data=DF_DOT, x="bregion", y="dot_product", hue="var_effect_12", kind="bar", aspect=1.5,
                  col="date", col_wrap=4, sharey=False)
rotateLabel(fig)
for ax in fig.axes.flatten():
    ax.axhline(0, color="k", alpha=0.5)
# savefig(fig, f"{SAVEDIR}/dot_across_ovar-catplot-2.pdf")


In [ ]:
# NEW CODE

In [ ]:
from pythonlib.tools.plottools import savefig
from pythonlib.tools.snstools import rotateLabel
from pythonlib.tools.pandastools import aggregGeneral

In [ ]:
SAVEDIR_PLOT = "/tmp"

In [ ]:
### (2) Angles (e.g,, dot product)
savedir = f"{SAVEDIR_PLOT}/angles"
os.makedirs(savedir, exist_ok=True)
DFANGLE = pd.concat(list_dfangle).reset_index(drop=True)
DFDOT = pd.concat(list_dfdot).reset_index(drop=True)
DFDOT[:2]
# Remove cases that are 2 levs and dot products. They are not defined.
a = DFANGLE["levs_exist_n"] == 2
b = DFANGLE["length_method"] == "dot"
DFANGLE = DFANGLE[~(a & b)]
assert sum(DFANGLE["norm_mean"].isna())==0
DFANGLE = append_col_with_grp_index(DFANGLE, ["length_method", "var_vector_length"], "length_method_var")
DFDOT = append_col_with_grp_index(DFDOT, ["length_method", "var_vector_length"], "length_method_var")

DFANGLE = append_col_with_grp_index(DFANGLE, ["var_effect", "vars_others"], "var_effect_others")

# Agg over (var_effect", "vars_others)
DFANGLE_AGG = aggregGeneral(DFANGLE, ["var_effect_others", "length_method_var", "var_effect", "vars_others", "bregion", "event", "animal", "date", "var_vector_length", "length_method", "levs_exist_n"], ["angle_mean", "norm_mean"])
# Agg over <var_other>
DFDOT_AGG = aggregGeneral(DFDOT, ["length_method_var", "var_effect_1", "var_effect_2", "bregion", "event", "var_effect_12", "animal", "date", "var_vector_length", "length_method", "min_levs_exist"], ["dot_product"])
### (1) ANGLE: Plot each date (older, more detailed plots)
if False: # TAkes too long
    from neuralmonkey.scripts.analy_syntax_good_eucl_trial import targeted_pca_euclidian_dist_angles_plots
    for date in DFANGLE["date"].unique():
        dfangle = DFANGLE[DFANGLE["date"] ==date].reset_index(drop=True)
        for var_vector_length in ["dist_yue_diff", "dist_norm"]:
            for length_method in ["sum", "dot"]:
                for min_levs_exist in [3, 2]:
                    savedir_this = f"{savedir}/angles_vectors/each_date/{date}-varlength={var_vector_length}-lengthmeth={length_method}-minlevs={min_levs_exist}"
                    os.makedirs(savedir_this, exist_ok=True)
                    print("Saving to: ", savedir_this)
                    targeted_pca_euclidian_dist_angles_plots(dfangle, var_vector_length, length_method, min_levs_exist, savedir_this)
                    plt.close("all")

### (2) ANGLE: Plot agg
fig = sns.catplot(data=DFANGLE_AGG, x="bregion", y="norm_mean", hue="var_effect_others", 
                row="length_method_var", col="levs_exist_n", kind="bar", aspect=1.5, sharey=False)
savefig(fig, f"{savedir}/ANGLE_AGG-1.pdf")

### (3) ANGLE: Plot each date
for levs_exist_n in DFANGLE["levs_exist_n"].unique():
    dfangle = DFANGLE[DFANGLE["levs_exist_n"] == levs_exist_n].reset_index(drop=True)

    fig = sns.catplot(data=dfangle, x="bregion", y="norm_mean", hue="var_effect_others", 
                    row="length_method_var", col="date", kind="bar", aspect=1.5, sharey=False)
    rotateLabel(fig)
    for ax in fig.axes.flatten():
        ax.axhline(0, color="k", alpha=0.5)
    savefig(fig, f"{savedir}/ANGLE_EACH-levs_exist_n={levs_exist_n}-catplot-1.pdf")

    fig = sns.catplot(data=dfangle, x="bregion", y="norm_mean", hue="var_effect_others", 
                    row="length_method_var", col="date", jitter=True, alpha=0.5, aspect=1.5, sharey=False)
    rotateLabel(fig)
    for ax in fig.axes.flatten():
        ax.axhline(0, color="k", alpha=0.5)
    savefig(fig, f"{savedir}/ANGLE_EACH-{levs_exist_n}-catplot-2.pdf")

    plt.close("all")

### (4) DOT: Plot each date
for date in DFDOT["date"].unique():
    dfdot = DFDOT[DFDOT["date"] == date].reset_index(drop=True)
    fig = sns.catplot(data=dfdot, x="bregion", y="dot_product", hue="var_effect_12", jitter=True, 
                    row="min_levs_exist", col="length_method_var", alpha=0.25, aspect=1.5)
    for ax in fig.axes.flatten():
        ax.axhline(0, color="k", alpha=0.5)
    savefig(fig, f"{savedir}/DOT_EACH-{date}-catplot-1.pdf")

    fig = sns.catplot(data=dfdot, x="bregion", y="dot_product", hue="var_effect_12", kind="boxen",
                    row="min_levs_exist", col="length_method_var", aspect=1.5, sharey=False)
    savefig(fig, f"{savedir}/DOT_EACH-{date}-catplot-2.pdf")

    fig = sns.catplot(data=dfdot, x="bregion", y="dot_product", hue="var_effect_12", kind="bar", errorbar="se", 
                    row="min_levs_exist", col="length_method_var", aspect=1.5, sharey=False)
    savefig(fig, f"{savedir}/DOT_EACH-{date}-catplot-3.pdf")

    plt.close("all")

### (5) DOT: Plot agg
fig = sns.catplot(data=DFDOT_AGG, x="bregion", y="dot_product", hue="var_effect_12", jitter=True, 
                row="min_levs_exist", col="length_method_var", alpha=0.25, aspect=1.5)
for ax in fig.axes.flatten():
    ax.axhline(0, color="k", alpha=0.5)
savefig(fig, f"{savedir}/DOT_AGG-catplot-1.pdf")
    
fig = sns.catplot(data=DFDOT_AGG, x="bregion", y="dot_product", hue="var_effect_12", kind="boxen",
                row="min_levs_exist", col="length_method_var", aspect=1.5, sharey=False)
savefig(fig, f"{savedir}/DOT_AGG-catplot-2.pdf")

fig = sns.catplot(data=DFDOT_AGG, x="bregion", y="dot_product", hue="var_effect_12", kind="bar", errorbar="se", 
                row="min_levs_exist", col="length_method_var", aspect=1.5, sharey=False)
savefig(fig, f"{savedir}/DOT_AGG-catplot-3.pdf")


plt.close("all")

for date in DFDOT["date"].unique():
    dfdot = DFDOT[DFDOT["date"] == date].reset_index(drop=True)
    fig = sns.catplot(data=dfdot, x="bregion", y="dot_product", hue="var_effect_12", jitter=True, 
                    row="min_levs_exist", col="length_method_var", alpha=0.25, aspect=1.5)
    for ax in fig.axes.flatten():
        ax.axhline(0, color="k", alpha=0.5)
    savefig(fig, f"{savedir}/DOT_EACH-{date}-catplot-1.pdf")

    fig = sns.catplot(data=dfdot, x="bregion", y="dot_product", hue="var_effect_12", kind="boxen",
                    row="min_levs_exist", col="length_method_var", aspect=1.5, sharey=False)
    savefig(fig, f"{savedir}/DOT_EACH-{date}-catplot-2.pdf")

    fig = sns.catplot(data=dfdot, x="bregion", y="dot_product", hue="var_effect_12", kind="bar", errorbar="se", 
                    row="min_levs_exist", col="length_method_var", aspect=1.5, sharey=False)
    savefig(fig, f"{savedir}/DOT_EACH-{date}-catplot-3.pdf")

    plt.close("all")


In [ ]:
DFDOT

In [ ]:
DFDOT_AGG

In [ ]:
fig = sns.catplot(data=DFDOT, x="bregion", y="dot_product", hue="var_effect_12", kind="bar", errorbar="se", 
                row="min_levs_exist", col="length_method_var", aspect=1.5, sharey=False)
rotateLabel(fig)
# savefig(fig, f"{savedir}/DOT_AGG-catplot-3.pdf")
fig = sns.catplot(data=DFDOT, x="bregion", y="dot_product", hue="var_effect_12", kind="boxen",
                row="min_levs_exist", col="length_method_var", aspect=1.5, sharey=False)
rotateLabel(fig)


# [Updated] Targeted PCA mult axes (coming back here after syntax state space)

In [ ]:
# Using the same procedure as there

In [ ]:
# During samp
DFallpa = DFallpa[:16]

from pythonlib.tools.vectools import average_vectors_wrapper, get_vector_from_angle

twind_scal = (0.2, 1.4)
from neuralmonkey.scripts.analy_syntax_good_eucl_trial import state_space_targeted_pca_scalar_single_one_axis_per_var, state_space_targeted_pca_scalar_single_one_var_mult_axes, targeted_pca_euclidian_dist_angles

variables = ["epoch", "FEAT_num_strokes_beh", "seqc_0_loc", "seqc_0_shape", "syntax_slot_0", "syntax_slot_1", "syntax_slot_2"]
variables_is_cat = [True, True, True, True, False, False, False]

list_subspaces = [
    ("syntax_slot_0", "syntax_slot_1"),
    # ("syntax_slot_1", "syntax_slot_2"),   
    # ("syntax_slot_0", "syntax_slot_2"),   
]

LIST_VAR_VAROTHERS_SS = [
    # ("syntax_slot_ratio", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "seqc_0_shape"]),
    # ("syntax_slot_ratio", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc"]),
    # ("syntax_slot_ratio", ["FEAT_num_strokes_beh", "epoch"]),

    # ("shapes_n_unique", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "seqc_0_shape"]),

    ("syntax_slot_0", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "seqc_0_shape", "syntax_slot_1"]),
    # ("syntax_slot_0", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "syntax_slot_1"]),
    # ("syntax_slot_0", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "seqc_0_shape"]),
    # ("syntax_slot_0", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc"]),
    ("syntax_slot_0", ["FEAT_num_strokes_beh", "epoch"]),

    ("syntax_slot_1", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "seqc_0_shape", "syntax_slot_0"]),
    # ("syntax_slot_1", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "syntax_slot_0"]),
    ("syntax_slot_1", ["FEAT_num_strokes_beh", "epoch"]),

    # ("syntax_slot_2", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "seqc_0_shape", "syntax_slot_0", "syntax_slot_1"]),
    # ("syntax_slot_2", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "seqc_0_shape", "syntax_slot_0"]),
    # ("syntax_slot_2", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "seqc_0_shape", "syntax_slot_1"]),
    # ("syntax_slot_2", ["FEAT_num_strokes_beh", "epoch"]),

    ("syntax_concrete", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "seqc_0_shape"]),
    ("syntax_concrete", ["FEAT_num_strokes_beh", "epoch"]),

    # ("FEAT_num_strokes_beh", ["epoch", "seqc_0_loc", "seqc_0_shape"]),
    # ("epoch", ["FEAT_num_strokes_beh", "seqc_0_loc", "seqc_0_shape"]),
    ("seqc_0_shape", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc"]),
]

LIST_DIMS = [(0,1), (1,2)]

LIST_VAR_VAROTHERS_REGR = [
    ("syntax_slot_0", ["syntax_slot_1", "seqc_0_loc", "seqc_0_shape", "epoch", "FEAT_num_strokes_beh"]),
    ("syntax_slot_1", ["syntax_slot_0", "seqc_0_loc", "seqc_0_shape", "epoch", "FEAT_num_strokes_beh"]),
    # ("FEAT_num_strokes_beh", ["epoch", "seqc_0_loc", "seqc_0_shape"]),
]
# else:
#     # TODO: find the most constrainig params givne this expt
#     LIST_VAR_VAROTHERS_REGR = [
#         ("syntax_slot_0", ["epoch", "seqc_0_loc", "seqc_0_shape", "syntax_slot_1"]),
#         ("syntax_slot_1", ["epoch", "seqc_0_loc", "seqc_0_shape", "syntax_slot_0"]),
#         # ("FEAT_num_strokes_beh", ["epoch", "seqc_0_loc", "seqc_0_shape"]),
#     ]


subspace_tuple = ('syntax_slot_0', 'syntax_slot_1')

SAVEDIR_ANALYSIS = f"/tmp/TERGET_PCA_EUCL/{animal}-{date}"
os.makedirs(SAVEDIR_ANALYSIS, exist_ok=True)
DFANGLE = state_space_targeted_pca_scalar_single_one_axis_per_var(DFallpa, SAVEDIR_ANALYSIS, twind_scal,
                                       variables, variables_is_cat, list_subspaces, LIST_VAR_VAROTHERS_SS, # For dim reduction and plotting state space
                                       subspace_tuple, LIST_VAR_VAROTHERS_REGR)


In [ ]:
twind_scal = (0.2, 1.4)

for _, row in DFallpa.iterrows():
    PA = row["pa"].copy()
    bregion = row["bregion"]
    event = row["event"]
    
    SAVEDIR = f"{SAVEDIR_ANALYSIS}/{event}-{bregion}"
    os.makedirs(SAVEDIR, exist_ok=True)

    ### Dim reductions, AND scalar state space plots
    # if subspace_filtdict:
    #     # Then take subset of data for computing the state space 
    just_extract_paredu = False
    LIST_DIMS = [(0,1)]
    # dict_subspace_pa, _, _, _, _ = state_space_targeted_pca_scalar_single(
    #                         PA, twind_scal, variables, variables_is_cat, list_subspaces, 
    #                         LIST_VAR_VAROTHERS_SS, LIST_DIMS, SAVEDIR, just_extract_paredu=just_extract_paredu)
    dict_subspace_pa, _, _, _, _ = state_space_targeted_pca_scalar_single_one_axis_per_var(
                            PA, twind_scal, variables, variables_is_cat, list_subspaces, 
                            LIST_VAR_VAROTHERS_SS, LIST_DIMS, SAVEDIR, just_extract_paredu=just_extract_paredu)

    asdda

In [ ]:
from pythonlib.tools.plottools import savefig
import pandas as pd

In [ ]:
SAVEDIR_ANALYSIS

In [ ]:
list_bregion = ["vlPFC"]
for _, row in DFallpa.iterrows():

    PA = row["pa"]
    bregion = row["bregion"]
    if bregion not in list_bregion:
        continue
    SAVEDIR = f"{SAVEDIR_ANALYSIS}/good/{bregion}"
    os.makedirs(SAVEDIR, exist_ok=True)

    ##### Update dim reduction
    variables_cont = []
    variables_cat = ["epoch", "FEAT_num_strokes_beh", "seqc_0_loc", "seqc_0_shape", "syntax_slot_0", "syntax_slot_1", "syntax_slot_2"]
    # var_subspace = "syntax_slot_0"
    var_subspace = variables_cat
    # variables_cat = ["epoch", "FEAT_num_strokes_beh", "seqc_0_loc", "seqc_0_shape"]
    # var_subspace = "seqc_0_loc"
    npcs_keep = 6
    tbin_dur = 0.2
    tbin_slide = 0.1
    LIST_DIMS = [(0,1), (2,3)]
    pa_subspace, subspace_axes_orig, subspace_axes_normed, dfcoeff, PAscalTest = state_space_targeted_pca_scalar_single_one_var_mult_axes(
                            PA, twind_scal, variables_cont, variables_cat, 
                            var_subspace, npcs_keep, 
                            LIST_VAR_VAROTHERS_SS, LIST_DIMS, SAVEDIR, 
                            savedir_pca_subspaces=SAVEDIR, 
                            tbin_dur = tbin_dur, tbin_slide = tbin_slide)

    if False:
        # Also plot using UMAP
        scalar_or_traj = "scal"
        dim_red_method = "umap"
        savedir = f"{SAVEDIR}/umap"
        os.makedirs(savedir, exist_ok=True)
        twind_pca = [-1, 1]

        Xredu, PAredu = pa_subspace.dataextract_dimred_wrapper(scalar_or_traj, dim_red_method, savedir, 
                                        twind_pca, umap_n_components=2, umap_n_neighbors=40,
                                        n_min_per_lev_lev_others = 2,
                                        return_pca_components=False)
        LIST_VAR_VAROTHERS_SS
        LIST_VAR = [x[0] for x in LIST_VAR_VAROTHERS_SS]
        LIST_VARS_OTHERS = [x[1] for x in LIST_VAR_VAROTHERS_SS]
        PAredu.plot_state_space_good_wrapper(savedir, LIST_VAR, LIST_VARS_OTHERS, 
                                            PLOT_CLEAN_VERSION = True,  
                                            nmin_trials_per_lev=None, 
                                            list_dim_timecourse=None, list_dims=None, 
                                            also_plot_heatmaps=False)
    ### Compute metrics

    ##### TODO: euclidean distance

    ##### Ordinal regression (including generalization)
    from neuralmonkey.scripts.analy_syntax_good_eucl_trial import ordinalregress_1_compute
    pa_subspace_this = pa_subspace.slice_by_dim_indices_wrapper("chans", list(range(npcs_keep)))
    DFCROSS, DFWITHIN = ordinalregress_1_compute(pa_subspace, LIST_VAR_VAROTHERS_REGR, SAVEDIR, nsplits=4,
                             apply_kernel = False, plot_regr=False)

    assert False
    
    # list_dfcross = []
    # list_dfwithin = []
    # for yvar, vars_grp in LIST_VAR_VAROTHERS_REGR:
    #     # yvar = "syntax_slot_0"
    #     # vars_grp = ["epoch", "FEAT_num_strokes_beh", "seqc_0_loc", "seqc_0_shape", "syntax_slot_1"]
    #     # yvar = "syntax_slot_1"
    #     # vars_grp = ["epoch", "FEAT_num_strokes_beh", "seqc_0_loc", "seqc_0_shape", "syntax_slot_0"]
    #     savedir = f"{SAVEDIR}/ordinal_regress-yvar={yvar}=varsgrp={vars_grp}"
    #     apply_kernel = False
    #     os.makedirs(savedir, exist_ok=True)
    #     npcs_keep = 6
    #     list_do_grid_search = [False]
    #     dfcross, dfwithin = kernel_ordinal_logistic_regression_wrapper(pa_subspace_this, yvar, vars_grp, savedir, 
    #                                             plot_test_data_projected=False, nsplits=4, PLOT=True, do_rezero=False,
    #                                             apply_kernel=apply_kernel, list_do_grid_search=list_do_grid_search)
    #     dfcross["yvar"] = yvar
    #     dfcross["vars_grp"] = [tuple(vars_grp) for _ in range(len(dfcross))]

    #     dfwithin["yvar"] = yvar
    #     dfwithin["vars_grp"] = [tuple(vars_grp) for _ in range(len(dfwithin))]

    #     list_dfcross.append(dfcross)
    #     list_dfwithin.append(dfwithin)

    # DFCROSS = pd.concat(list_dfcross).reset_index(drop=True)
    # DFWITHIN = pd.concat(list_dfwithin).reset_index(drop=True)

    # # Compare the angles of the regression axes.
    # list_coeff = []
    # for _, row in DFWITHIN.iterrows():
    #     coeff = row["res"]["coeff"]
    #     list_coeff.append(coeff)
    # DFWITHIN["coeff"] = list_coeff
    # coeff_mat = np.stack(DFWITHIN["coeff"])

    # labels_row = DFWITHIN["grp"].tolist()
    # labels_row = [tuple(x) for x in DFWITHIN.loc[:, ["yvar", "grp"]].values.tolist()]
    
    # from pythonlib.tools.snstools import heatmap_mat
    # fig, ax, rgba_values = heatmap_mat(coeff_mat, diverge=True, labels_row=labels_row, annotate_heatmap=False)
    # savefig(fig, f"{SAVEDIR}/ordinal_regress-coefficients_heatmap.pdf")

    # pd.to_pickle(DFCROSS, f"{SAVEDIR}/DFCROSS.pkl")
    # pd.to_pickle(DFWITHIN, f"{SAVEDIR}/DFWITHIN.pkl")


In [ ]:
##### Compare angles of regression coefficients across all conditions.
from neuralmonkey.scripts.analy_syntax_good_eucl_trial import ordinalregress_2_regr_coeff_pairs
savedir = f"{SAVEDIR}/pairwise_regr_coeffs"
os.makedirs(savedir, exist_ok=True)
ordinalregress_2_regr_coeff_pairs(DFWITHIN, savedir=savedir)

In [ ]:
var_effect = LIST_VAR_VAROTHERS_REGR[0][0]
vars_others = LIST_VAR_VAROTHERS_REGR[0][1]
### (2) Compute euclidian distances
from neuralmonkey.analyses.euclidian_distance import timevarying_compute_fast_to_scalar
savedir = "/tmp"
prune_levs_min_n_trials = 2
dfdist, _ = timevarying_compute_fast_to_scalar(pa_subspace, [var_effect] + vars_others, 
                                                plot_conjunctions_savedir=savedir,
                                                prune_levs_min_n_trials=prune_levs_min_n_trials,
                                                get_only_one_direction=False)

### (3) Get angles between all conditions
from neuralmonkey.analyses.euclidian_distance import compute_angle_between_conditions
dfangle = compute_angle_between_conditions(pa_subspace, dfdist, var_effect, vars_others)
assert np.all(dfangle["labels_1"] == dfdist["labels_1"])
assert np.all(dfangle["labels_2"] == dfdist["labels_2"])
dfdist["theta"] = dfangle["theta"]
dfdist["norm"] = dfangle["norm"]
# dfdist["vector"] = dfangle["vector"]  

from neuralmonkey.analyses.euclidian_distance import dfdist_variables_generate_constrast_strings, dfdist_variables_generate_var_same
var_same = dfdist_variables_generate_var_same([var_effect] + vars_others)
var_same_val = dfdist_variables_generate_constrast_strings([var_effect] + vars_others, contrasts_diff=[var_effect], contrasts_either=[])[0]
from neuralmonkey.analyses.euclidian_distance import compute_average_angle_between_pairs_of_levels_of_vareffect
dfanglemean = compute_average_angle_between_pairs_of_levels_of_vareffect(dfdist, var_effect)

In [ ]:
##### Cleaning up DFWITHIN


In [ ]:

# Merging dfwithin and dfcross 
# [ignore?]
DFCROSS = DFCROSS[DFCROSS["grp_train"] != DFCROSS["grp_test"]].reset_index(drop=True)
DFWITHIN["grp_train"] = DFWITHIN["grp"]
DFWITHIN["grp_test"] = DFWITHIN["grp"] 

DFWITHIN["var_effect"] = DFWITHIN["yvar"] 
DFWITHIN["vars_others"] = DFWITHIN["vars_grp"] 
DFCROSS["var_effect"] = DFCROSS["yvar"] 
DFCROSS["vars_others"] = DFCROSS["vars_grp"] 

DFWITHIN["res"] = "none"

DFRES = pd.concat([DFWITHIN, DFCROSS], axis=0).reset_index(drop=True)
DFRES = DFRES.drop(["grp"], axis=1)
from pythonlib.cluster.clustclass import generate_clustclass_from_flat_df_rectangle

# This fails -- it requires square data
Cl = generate_clustclass_from_flat_df_rectangle(DFRES, "grp_train", "grp_test", var_value="balanced_accuracy_adjusted", 
                                      var_labels=vars_grp)
fig, _ = Cl.rsa_plot_heatmap()
savefig(fig, f"{plot_savedir}/heatmap-r2_test.pdf")


In [ ]:
##### dot product of vectors.


In [ ]:
DFallpa

In [ ]:
##### Load all regression results and summarize across regions
path = "/tmp/TERGET_PCA_EUCL/Diego-230913/good/vlPFC/DFWITHIN.pkl"
# path = "/tmp/TERGET_PCA_EUCL/Diego-230913/good/M1/DFWITHIN.pkl"

# path = "/tmp/TERGET_PCA_EUCL/Diego-230726/good/M1/DFWITHIN.pkl"
path = "/tmp/TERGET_PCA_EUCL/Diego-230726/good/vlPFC/DFWITHIN.pkl"
dfwithin = pd.read_pickle(path)



In [ ]:

# First get dim reduced data
list_bregion = ["M1", "vlPFC"]
list_pa_redu = []
for _, row in DFallpa.iterrows():

    PA = row["pa"]
    bregion = row["bregion"]

    print("Getting this bregion: ", bregion)
    
    ##### Update dim reduction
    variables_cont = []
    variables_cat = ["epoch", "FEAT_num_strokes_beh", "seqc_0_loc", "seqc_0_shape", "syntax_slot_0", "syntax_slot_1", "syntax_slot_2"]
    # var_subspace = "syntax_slot_0"
    var_subspace = variables_cat
    # variables_cat = ["epoch", "FEAT_num_strokes_beh", "seqc_0_loc", "seqc_0_shape"]
    # var_subspace = "seqc_0_loc"
    npcs_keep = 6
    tbin_dur = 0.2
    tbin_slide = 0.1
    LIST_DIMS = [(0,1), (2,3)]
    pa_subspace, subspace_axes_orig, subspace_axes_normed, dfcoeff, PAscalTest = state_space_targeted_pca_scalar_single_one_var_mult_axes(
                            PA, twind_scal, variables_cont, variables_cat, 
                            var_subspace, npcs_keep, 
                            LIST_VAR_VAROTHERS_SS, LIST_DIMS, SAVEDIR, 
                            just_extract_paredu=True,
                            savedir_pca_subspaces=SAVEDIR, 
                            tbin_dur = tbin_dur, tbin_slide = tbin_slide)

    list_pa_redu.append(pa_subspace)

    plt.close("all")

In [ ]:
DFallpa["pa_redu"] = list_pa_redu

In [ ]:
DFallpa

In [ ]:
var_effect = LIST_VAR_VAROTHERS_REGR[0][0]
vars_others = LIST_VAR_VAROTHERS_REGR[0][1]
ndims = 4
N_MIN_TRIALS = 3

In [ ]:
DFWITHIN["grp_train"] = DFWITHIN["grp"]
DFWITHIN["grp_test"] = DFWITHIN["grp"] 


In [ ]:
from neuralmonkey.scripts.analy_syntax_good_eucl_trial import regression_fit_and_score_single_ordinal
bregion = "preSMA"
event = "03_samp"

plot_savedir = "/tmp/TEST"
os.makedirs(plot_savedir, exist_ok=True)

# list_dfcross = []
# list_dfwithin = []
# for yvar, vars_grp in LIST_VAR_VAROTHERS_REGR:
#     # yvar = "syntax_slot_0"
#     # vars_grp = ["epoch", "FEAT_num_strokes_beh", "seqc_0_loc", "seqc_0_shape", "syntax_slot_1"]
#     # yvar = "syntax_slot_1"
#     # vars_grp = ["epoch", "FEAT_num_strokes_beh", "seqc_0_loc", "seqc_0_shape", "syntax_slot_0"]
#     savedir = f"{SAVEDIR}/ordinal_regress-yvar={yvar}=varsgrp={vars_grp}"
#     apply_kernel = False
#     os.makedirs(savedir, exist_ok=True)
#     npcs_keep = 6
#     list_do_grid_search = [False]
#     pa_subspace_this = pa_subspace.slice_by_dim_indices_wrapper("chans", list(range(npcs_keep)))

regression_fit_and_score_single_ordinal(DFallpa, bregion, event, var_effect, 
                                        vars_others, ndims, N_MIN_TRIALS, plot_savedir)

### [GOOD CODE] wraps everything above

In [ ]:
SAVEDIR_ANALYSIS = f"/tmp/SYNTAX_TRIAL/{animal}-{date}"
os.makedirs(SAVEDIR_ANALYSIS, exist_ok=True)

In [ ]:
from neuralmonkey.scripts.analy_syntax_good_eucl_trial import targeted_pca_combined_v2_good
LIST_VAR_VAROTHERS_SS = [
    # ("syntax_slot_ratio", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "seqc_0_shape"]),
    # ("syntax_slot_ratio", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc"]),
    # ("syntax_slot_ratio", ["FEAT_num_strokes_beh", "epoch"]),

    # ("shapes_n_unique", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "seqc_0_shape"]),

    ("syntax_slot_0", ["seqc_0_loc", "seqc_0_shape", "syntax_slot_1", "epoch", "FEAT_num_strokes_beh"]),
    ("syntax_slot_0", ["seqc_0_loc", "seqc_0_shape", "syntax_slot_1", "epoch"]),
    ("syntax_slot_0", ["seqc_0_loc", "seqc_0_shape", "syntax_slot_1"]),
    ("syntax_slot_0", ["seqc_0_loc", "seqc_0_shape"]),
    ("syntax_slot_0", ["FEAT_num_strokes_beh", "epoch"]),

    ("syntax_slot_1", ["seqc_0_loc", "seqc_0_shape", "syntax_slot_0", "epoch", "FEAT_num_strokes_beh"]),
    ("syntax_slot_1", ["seqc_0_loc", "seqc_0_shape", "syntax_slot_0", "epoch"]),
    ("syntax_slot_1", ["seqc_0_loc", "seqc_0_shape", "syntax_slot_0"]),
    ("syntax_slot_1", ["seqc_0_loc", "seqc_0_shape"]),
    ("syntax_slot_1", ["FEAT_num_strokes_beh", "epoch"]),

    # ("syntax_slot_2", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "seqc_0_shape", "syntax_slot_0", "syntax_slot_1"]),
    # ("syntax_slot_2", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "seqc_0_shape", "syntax_slot_0"]),
    # ("syntax_slot_2", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "seqc_0_shape", "syntax_slot_1"]),
    # ("syntax_slot_2", ["FEAT_num_strokes_beh", "epoch"]),

    # ("syntax_concrete", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc", "seqc_0_shape"]),
    ("syntax_concrete", ["FEAT_num_strokes_beh", "epoch"]),

    ("FEAT_num_strokes_beh", ["epoch", "seqc_0_loc", "seqc_0_shape"]),
    # ("epoch", ["FEAT_num_strokes_beh", "seqc_0_loc", "seqc_0_shape"]),
    ("seqc_0_shape", ["FEAT_num_strokes_beh", "epoch", "seqc_0_loc"]),
]
targeted_pca_combined_v2_good(DFallpa, SAVEDIR_ANALYSIS, LIST_VAR_VAROTHERS_SS, DEBUG=True)

### [MULT] [NEW] Load and plot all data

In [3]:
# For plots related to angles (dot product) see above
# For other plots, see below.


In [4]:
from neuralmonkey.classes.session import _REGIONS_IN_ORDER_COMBINED

from pythonlib.tools.pandastools import grouping_append_and_return_inner_items_good
from pythonlib.tools.vectools import get_vector_from_angle
from pythonlib.tools.plottools import savefig
from pythonlib.tools.snstools import rotateLabel


In [ ]:
SAVEDIR = "/lemur2/lucas/analyses/recordings/main/syntax_good_trial/targeted_pca_v2"
list_regions = _REGIONS_IN_ORDER_COMBINED
event ="03_samp"

### Collect all data
for animal in ["Diego", "Pancho"]:
    if animal=="Diego":
        list_dates = [230728, 231118, 240822, 230723, 230724, 230726, 230727, 230815, 230816, 230817, 230913, 230914, 230915, 231116, 250321]
        # list_dates = [230728, 230723, 230724, 230726, 230727, 230815, 230816, 230817, 230913, 230914, 230915]
        # list_dates = [230728, 230723, 230724, 230726, 230727]
        # animal = "Diego"
    elif animal=="Pancho":
        list_dates = [230810, 230811, 230824, 230826, 230829, 231114, 231116, 240830, 220831, 220901, 250321, 250322, 220902, 220906, 220907, 220908, 220909]
        # list_dates = [230810, 230811, 230824, 230826, 230829, 231114, 231116, 240830, 220901, 250321, 250322, 220902, 220906, 220907, 220908, 220909]
        # list_dates = [220901, 250321, 220902, 220906, 220907, 220908, 220909]
        # list_dates = [220901, 250321, 220902, 220906, 220907, 220908, 220909]
        # animal = "Pancho"
    else:
        assert False

    list_dfangle = []
    list_dfdot = []
    list_dfdist = []
    LIST_DFWITHIN = []
    LIST_DFCROSS = []
    for date in list_dates:
        for bregion in list_regions:
            # EUCLIDEAN
            path = f"{SAVEDIR}/{animal}-{date}-q=RULE_BASE_trial/{event}-{bregion}/DFDIST.pkl"
            if os.path.exists(path):
                dfdist = pd.read_pickle(path)
                dfdist["var_subspace"] = [tuple(x) if isinstance(x, list) else x for x in dfdist["var_subspace"]]
                if "i_proj" not in dfdist:
                    dfdist["i_proj"] = 0
                for _df in [dfdist]:
                    _df["date"] = date
                    _df["bregion"] = bregion
                list_dfdist.append(dfdist)

                # REGRESSION
                path = f"{SAVEDIR}/{animal}-{date}-q=RULE_BASE_trial/{event}-{bregion}/DFWITHIN.pkl"
                dfwithin = pd.read_pickle(path)
                
                path = f"{SAVEDIR}/{animal}-{date}-q=RULE_BASE_trial/{event}-{bregion}/DFCROSS.pkl"
                dfcross = pd.read_pickle(path)


                from neuralmonkey.scripts.analy_syntax_good_eucl_state import kernel_ordinal_logistic_regression_wrapper_postprocess_mult_varsgrp
                dfcross, dfwithin = kernel_ordinal_logistic_regression_wrapper_postprocess_mult_varsgrp(dfcross, dfwithin)

                # Also load dfdist to extract the name of the subspace
                if "var_conj" not in dfdist:
                    dfdist["var_conj"] = "none"
                    dfdist["var_conj_lev"] = "none"
                # - confirm there's only one subspace
                subspace_tuple = list(set([tuple(x) for x in dfdist.loc[:, ["var_subspace", "var_conj", "var_conj_lev"]].values.tolist()]))
                assert len(subspace_tuple)==1
                # - then prune dfdist to make the following faster
                dfdist = dfdist.iloc[:1].reset_index(drop=True)
                dfdist["var_subspace"] = [tuple(x)  if isinstance(x, list) else x for x in dfdist["var_subspace"]]  
                dfdist = append_col_with_grp_index(dfdist, ["var_subspace", "var_conj", "var_conj_lev"], "subspace")
                
                tmp = dfdist["subspace"].unique().tolist()
                subspace = tmp[0]

                tmp = dfdist["i_proj"].unique().tolist()
                i_proj = tmp[0]

                for _df in [dfcross, dfwithin]:
                    _df["bregion"] = bregion
                    _df["date"] = date
                    _df["subspace"] = subspace
                    _df["i_proj"] = i_proj


                for _df in [dfwithin, dfcross]:
                    _df["date"] = date
                    _df["bregion"] = bregion

                LIST_DFWITHIN.append(dfwithin)
                LIST_DFCROSS.append(dfcross)

                # ANGLES
                path = f"{SAVEDIR}/{animal}-{date}-q=RULE_BASE_trial/{event}-{bregion}/DFANGLE.pkl"
                dfangle = pd.read_pickle(path)
                dfangle["animal"] = animal
                dfangle["date"] = date
                dfangle["bregion"] = bregion

                # Also compute dfdot (consistency across vars_others)
                from neuralmonkey.scripts.analy_syntax_good_eucl_trial import targeted_pca_euclidian_dist_angles_compute_dfdot
                for var_vector_length in ["dist_yue_diff"]:
                    for length_method in ["sum", "dot"]:
                        for min_levs_exist in [2, 3]:
                            
                            if min_levs_exist==2 and length_method=="dot": 
                                # This doestn exist
                                continue
                            
                            dfdot = targeted_pca_euclidian_dist_angles_compute_dfdot(dfangle, var_vector_length, 
                                                                                    length_method, min_levs_exist)
                            if dfdot is None:
                                print("skipping: ", var_vector_length, length_method, min_levs_exist)
                            else:
                                dfdot["animal"] = animal
                                dfdot["date"] = date
                                dfdot["bregion"] = bregion
                                list_dfdot.append(dfdot)
                            
                list_dfangle.append(dfangle)
            
    SAVEDIR_PLOT = f"{SAVEDIR}/MULT/{animal}"
    os.makedirs(SAVEDIR_PLOT, exist_ok=True)

    #####################################################
    ### (1) Plot euclidean distance
    savedir = f"{SAVEDIR_PLOT}/euclidean"
    os.makedirs(savedir, exist_ok=True)
    from pythonlib.tools.pandastools import grouping_append_and_return_inner_items_good
    from pythonlib.tools.pandastools import aggregGeneral
    from neuralmonkey.analyses.euclidian_distance import dfdist_variables_generate_var_same
    from neuralmonkey.analyses.euclidian_distance import dfdist_variables_generate_var_same
    from neuralmonkey.neuralplots.brainschematic import datamod_reorder_by_bregion

    # Collect data
    DFDIST = pd.concat(list_dfdist).reset_index(drop=True)

    # Split into different varsets (makes it easier to analyse)
    grpdict = grouping_append_and_return_inner_items_good(DFDIST, ["var_idx", "var_effect", "vars_others"])
    MAP_VARSET_TO_DFDIST = {}
    for grp, inds in grpdict.items():
        MAP_VARSET_TO_DFDIST[grp] = DFDIST.iloc[inds].reset_index(drop=True)
    # one datapt per each (date, varsame)
    MAP_VARSET_TO_DFDISTAGG = {}
    for varset, dfdist in MAP_VARSET_TO_DFDIST.items():
        var_effect = varset[1]
        vars_others = list(varset[2])
        varsame = dfdist_variables_generate_var_same([var_effect] + vars_others)
        dfdistagg = aggregGeneral(dfdist, [varsame, "bregion", "date"], ["dist_yue_diff"])
        dfdistagg = datamod_reorder_by_bregion(dfdistagg)
        MAP_VARSET_TO_DFDISTAGG[varset] = dfdistagg

    for varset, dfdist in MAP_VARSET_TO_DFDIST.items():
        print(varset)

        var_effect = varset[1]
        vars_others = list(varset[2])
        varsame = dfdist_variables_generate_var_same([var_effect] + vars_others)

        # Split by dates
        # sns.catplot(data=dfdist, x="bregion", y="dist_yue_diff", col="date", col_wrap=6, hue=varsame, height=8,
        #             jitter=True, alpha=0.3)    
        fig = sns.catplot(data=dfdist, x="bregion", y="dist_yue_diff", col="date", col_wrap=6, hue=varsame, height=8,
                    kind="boxen")
        savefig(fig, f"{savedir}/vareffect={var_effect}-varsother={vars_others}-catplot-1.pdf")
        
        fig = sns.catplot(data=dfdist, x="bregion", y="dist_yue_diff", col="date", col_wrap=6, hue=varsame, height=8,
                    kind="bar", errorbar="se")
        savefig(fig, f"{savedir}/vareffect={var_effect}-varsother={vars_others}-catplot-2.pdf")
        
        # agg, datapt= (date, varsame)
        dfdistagg = MAP_VARSET_TO_DFDISTAGG[varset]
        fig = sns.catplot(data=dfdistagg, x="bregion", y="dist_yue_diff", hue=varsame, height=8, kind="boxen")
        savefig(fig, f"{savedir}/vareffect={var_effect}-varsother={vars_others}-catplot-agg-1.pdf")

        fig = sns.catplot(data=dfdistagg, x="bregion", y="dist_yue_diff", hue=varsame, height=8,
                    kind="bar", errorbar="se")
        savefig(fig, f"{savedir}/vareffect={var_effect}-varsother={vars_others}-catplot-agg-2.pdf")

        plt.close("all")

    ##########################################################
    ### (2) Angles (e.g,, dot product)
    savedir = f"{SAVEDIR_PLOT}/angles"
    os.makedirs(savedir, exist_ok=True)
    DFANGLE = pd.concat(list_dfangle).reset_index(drop=True)
    DFDOT = pd.concat(list_dfdot).reset_index(drop=True)
    DFDOT[:2]
    # Remove cases that are 2 levs and dot products. They are not defined.
    a = DFANGLE["levs_exist_n"] == 2
    b = DFANGLE["length_method"] == "dot"
    DFANGLE = DFANGLE[~(a & b)]
    assert sum(DFANGLE["norm_mean"].isna())==0
    DFANGLE = append_col_with_grp_index(DFANGLE, ["length_method", "var_vector_length"], "length_method_var")
    DFDOT = append_col_with_grp_index(DFDOT, ["length_method", "var_vector_length"], "length_method_var")

    DFANGLE = append_col_with_grp_index(DFANGLE, ["var_effect", "vars_others"], "var_effect_others")

    # Agg over (var_effect", "vars_others)
    DFANGLE_AGG = aggregGeneral(DFANGLE, ["var_effect_others", "length_method_var", "var_effect", "vars_others", "bregion", "event", "animal", "date", "var_vector_length", "length_method", "levs_exist_n"], ["angle_mean", "norm_mean"])
    # Agg over <var_other>
    DFDOT_AGG = aggregGeneral(DFDOT, ["length_method_var", "var_effect_1", "var_effect_2", "bregion", "event", "var_effect_12", "animal", "date", "var_vector_length", "length_method", "min_levs_exist"], ["dot_product"])
    ### (1) ANGLE: Plot each date (older, more detailed plots)
    if False: # TAkes too long
        from neuralmonkey.scripts.analy_syntax_good_eucl_trial import targeted_pca_euclidian_dist_angles_plots
        for date in DFANGLE["date"].unique():
            dfangle = DFANGLE[DFANGLE["date"] ==date].reset_index(drop=True)
            for var_vector_length in ["dist_yue_diff", "dist_norm"]:
                for length_method in ["sum", "dot"]:
                    for min_levs_exist in [3, 2]:
                        savedir_this = f"{savedir}/angles_vectors/each_date/{date}-varlength={var_vector_length}-lengthmeth={length_method}-minlevs={min_levs_exist}"
                        os.makedirs(savedir_this, exist_ok=True)
                        print("Saving to: ", savedir_this)
                        targeted_pca_euclidian_dist_angles_plots(dfangle, var_vector_length, length_method, min_levs_exist, savedir_this)
                        plt.close("all")

    ### (2) ANGLE: Plot agg
    fig = sns.catplot(data=DFANGLE_AGG, x="bregion", y="norm_mean", hue="var_effect_others", 
                    row="length_method_var", col="levs_exist_n", kind="bar", aspect=1.5, sharey=False)
    savefig(fig, f"{savedir}/ANGLE_AGG-1.pdf")

    ### (3) ANGLE: Plot each date
    for levs_exist_n in DFANGLE["levs_exist_n"].unique():
        dfangle = DFANGLE[DFANGLE["levs_exist_n"] == levs_exist_n].reset_index(drop=True)

        fig = sns.catplot(data=dfangle, x="bregion", y="norm_mean", hue="var_effect_others", 
                        row="length_method_var", col="date", kind="bar", aspect=1.5, sharey=False)
        rotateLabel(fig)
        for ax in fig.axes.flatten():
            ax.axhline(0, color="k", alpha=0.5)
        savefig(fig, f"{savedir}/ANGLE_EACH-levs_exist_n={levs_exist_n}-catplot-1.pdf")

        fig = sns.catplot(data=dfangle, x="bregion", y="norm_mean", hue="var_effect_others", 
                        row="length_method_var", col="date", jitter=True, alpha=0.5, aspect=1.5, sharey=False)
        rotateLabel(fig)
        for ax in fig.axes.flatten():
            ax.axhline(0, color="k", alpha=0.5)
        savefig(fig, f"{savedir}/ANGLE_EACH-{levs_exist_n}-catplot-2.pdf")

        plt.close("all")

    ### (4) DOT: Plot each date
    for date in DFDOT["date"].unique():
        dfdot = DFDOT[DFDOT["date"] == date].reset_index(drop=True)
        fig = sns.catplot(data=dfdot, x="bregion", y="dot_product", hue="var_effect_12", jitter=True, 
                        row="min_levs_exist", col="length_method_var", alpha=0.25, aspect=1.5)
        for ax in fig.axes.flatten():
            ax.axhline(0, color="k", alpha=0.5)
        savefig(fig, f"{savedir}/DOT_EACH-{date}-catplot-1.pdf")

        fig = sns.catplot(data=dfdot, x="bregion", y="dot_product", hue="var_effect_12", kind="boxen",
                        row="min_levs_exist", col="length_method_var", aspect=1.5, sharey=False)
        savefig(fig, f"{savedir}/DOT_EACH-{date}-catplot-2.pdf")

        fig = sns.catplot(data=dfdot, x="bregion", y="dot_product", hue="var_effect_12", kind="bar", errorbar="se", 
                        row="min_levs_exist", col="length_method_var", aspect=1.5, sharey=False)
        savefig(fig, f"{savedir}/DOT_EACH-{date}-catplot-3.pdf")

        plt.close("all")

    ### (5) DOT: Plot agg
    fig = sns.catplot(data=DFDOT_AGG, x="bregion", y="dot_product", hue="var_effect_12", jitter=True, 
                    row="min_levs_exist", col="length_method_var", alpha=0.25, aspect=1.5)
    for ax in fig.axes.flatten():
        ax.axhline(0, color="k", alpha=0.5)
    savefig(fig, f"{savedir}/DOT_AGG-catplot-1.pdf")
        
    fig = sns.catplot(data=DFDOT_AGG, x="bregion", y="dot_product", hue="var_effect_12", kind="boxen",
                    row="min_levs_exist", col="length_method_var", aspect=1.5, sharey=False)
    savefig(fig, f"{savedir}/DOT_AGG-catplot-2.pdf")

    fig = sns.catplot(data=DFDOT_AGG, x="bregion", y="dot_product", hue="var_effect_12", kind="bar", errorbar="se", 
                    row="min_levs_exist", col="length_method_var", aspect=1.5, sharey=False)
    savefig(fig, f"{savedir}/DOT_AGG-catplot-3.pdf")


    plt.close("all")

    for date in DFDOT["date"].unique():
        dfdot = DFDOT[DFDOT["date"] == date].reset_index(drop=True)
        fig = sns.catplot(data=dfdot, x="bregion", y="dot_product", hue="var_effect_12", jitter=True, 
                        row="min_levs_exist", col="length_method_var", alpha=0.25, aspect=1.5)
        for ax in fig.axes.flatten():
            ax.axhline(0, color="k", alpha=0.5)
        savefig(fig, f"{savedir}/DOT_EACH-{date}-catplot-1.pdf")

        fig = sns.catplot(data=dfdot, x="bregion", y="dot_product", hue="var_effect_12", kind="boxen",
                        row="min_levs_exist", col="length_method_var", aspect=1.5, sharey=False)
        savefig(fig, f"{savedir}/DOT_EACH-{date}-catplot-2.pdf")

        fig = sns.catplot(data=dfdot, x="bregion", y="dot_product", hue="var_effect_12", kind="bar", errorbar="se", 
                        row="min_levs_exist", col="length_method_var", aspect=1.5, sharey=False)
        savefig(fig, f"{savedir}/DOT_EACH-{date}-catplot-3.pdf")

        plt.close("all")

    ############################################
    ### Regression
    savedir = f"{SAVEDIR_PLOT}/regression"
    os.makedirs(savedir, exist_ok=True)
    DFWITHIN = pd.concat(LIST_DFWITHIN).reset_index(drop=True)
    DFCROSS = pd.concat(LIST_DFCROSS).reset_index(drop=True)
    ############################################
    ### Postprocess        
    from neuralmonkey.scripts.analy_syntax_good_eucl_state import kernel_ordinal_logistic_regression_wrapper_CONCATED_postprocess, kernel_ordinal_logistic_regression_wrapper_CONCATED_plot_all
    vars_datapt = ["epoch", "FEAT_num_strokes_beh", "syntax_slot_0", "syntax_slot_1", "seqc_0_loc", "seqc_0_shape"]
    DFCROSS, DFWITHIN, DFWITHIN_AGG_SHP, DFWITHIN_AGG_DATE = kernel_ordinal_logistic_regression_wrapper_CONCATED_postprocess(DFCROSS, DFWITHIN, vars_datapt)

    # savedir = "/tmp"
    kernel_ordinal_logistic_regression_wrapper_CONCATED_plot_all(DFCROSS, DFWITHIN, DFWITHIN_AGG_SHP, DFWITHIN_AGG_DATE, savedir)


Making copy and replacing None with 'none', for:  balanced_accuracy
* This adds compute time!!!!
Making copy and replacing None with 'none', for:  balanced_accuracy_adjusted
* This adds compute time!!!!
Making copy and replacing None with 'none', for:  balanced_accuracy
* This adds compute time!!!!
Making copy and replacing None with 'none', for:  balanced_accuracy_adjusted
* This adds compute time!!!!


NameError: name 'asd' is not defined

In [29]:
a = dfdist["var_effect"] == "syntax_slot_0"
b = dfdist["vars_others"] == ('seqc_0_loc', 'seqc_0_shape', 'syntax_slot_1', 'epoch', 'FEAT_num_strokes_beh')
dfdist = dfdist[a & b]

In [30]:
dfdist["var_effect"].unique()

array(['syntax_slot_0'], dtype=object)

In [ ]:
dfdist = dfdist.reset_index(drop=True)

array([('seqc_0_loc', 'seqc_0_shape', 'syntax_slot_1', 'epoch', 'FEAT_num_strokes_beh')],
      dtype=object)

In [70]:
from neuralmonkey.analyses.euclidian_distance import compute_average_angle_between_pairs_of_levels_of_vareffect
var_effect = "syntax_slot_0"
vars_others = "('seqc_0_loc', 'seqc_0_shape', 'syntax_slot_1', 'epoch', 'FEAT_num_strokes_beh')"
compute_average_angle_between_pairs_of_levels_of_vareffect(dfdist, var_effect)

,var_other,levs_exist,angles,weights,angle_mean,norm_mean,var_vector_length,length_method
0,"((-1, 1), line-6-2-0, 1, llCV2, 5)","[1, 2]",[4.832475415954616],[0.5906494632238395],4.832475,0.590649,dist_norm,sum
1,"((-1, 1), line-6-2-0, 1, llCV2, 5)","[1, 2]",[4.832475415954616],[0.5906494632238395],NaN,NaN,dist_norm,dot
2,"((-1, 1), line-6-2-0, 1, llCV2, 5)","[1, 2]",[4.832475415954616],[0.033720521171148654],4.832475,0.033721,dist_yue_diff,sum
3,"((-1, 1), line-6-2-0, 1, llCV2, 5)","[1, 2]",[4.832475415954616],[0.033720521171148654],NaN,NaN,dist_yue_diff,dot
4,"((-1, 1), line-6-2-0, 2, llCV2, 5)","[1, 2]",[3.7363294138416516],[0.530802185762632],3.736329,0.530802,dist_norm,sum
5,"((-1, 1), line-6-2-0, 2, llCV2, 5)","[1, 2]",[3.7363294138416516],[0.530802185762632],NaN,NaN,dist_norm,dot
6,"((-1, 1), line-6-2-0, 2, llCV2, 5)","[1, 2]",[3.7363294138416516],[-0.007550998155462896],0.594737,0.007551,dist_yue_diff,sum
7,"((-1, 1), line-6-2-0, 2, llCV2, 5)","[1, 2]",[3.7363294138416516],[-0.007550998155462896],NaN,NaN,dist_yue_diff,dot
8,"((0, 0), line-6-2-0, 0, llCV2, 5)","[2, 3]",[5.821231741264794],[0.5186904284585455],5.821232,0.518690,dist_norm,sum
9,"((0, 0), line-6-2-0, 0, llCV2, 5)","[2, 3]",[5.821231741264794],[0.5186904284585455],NaN,NaN,dist_norm,dot


In [6]:
dfdist

,labels_1,labels_2,dist_mean,n_1_2,DIST_50,DIST_98,dist_norm,dist_yue_diff,dist_within_norm,syntax_slot_0_1,syntax_slot_0_2,syntax_slot_0_12,syntax_slot_0_same,seqc_0_loc_1,seqc_0_loc_2,seqc_0_loc_12,seqc_0_loc_same,seqc_0_shape_1,seqc_0_shape_2,seqc_0_shape_12,seqc_0_shape_same,syntax_slot_1_1,syntax_slot_1_2,syntax_slot_1_12,syntax_slot_1_same,epoch_1,epoch_2,epoch_12,epoch_same,FEAT_num_strokes_beh_1,FEAT_num_strokes_beh_2,FEAT_num_strokes_beh_12,FEAT_num_strokes_beh_same,same-syntax_slot_0|seqc_0_loc|seqc_0_shape|syntax_slot_1|epoch|FEAT_num_strokes_beh,data_dim,theta,norm,vars_others_1,vars_others_2,vars_others_same,bregion,event,var_subspace,var_idx,var_effect,vars_others,same-syntax_slot_1|seqc_0_loc|seqc_0_shape|syntax_slot_0|epoch|FEAT_num_strokes_beh,i_proj,date
0,"(1, (-1, 1), line-6-2-0, 1, llCV2, 5)","(1, (-1, 1), line-6-2-0, 1, llCV2, 5)",1.572092,"(11, 11)",1.610711,2.772302,0.567071,0.000000,0.567071,1,1,1|1,True,"(-1, 1)","(-1, 1)","(-1, 1)|(-1, 1)",True,line-6-2-0,line-6-2-0,line-6-2-0|line-6-2-0,True,1,1,1|1,True,llCV2,llCV2,llCV2|llCV2,True,5,5,5|5,True,1|1|1|1|1|1,6,0.000000,0.000000,"((-1, 1), line-6-2-0, 1, llCV2, 5)","((-1, 1), line-6-2-0, 1, llCV2, 5)",True,M1,03_samp,"(epoch, FEAT_num_strokes_beh, seqc_0_loc, seqc...",0,syntax_slot_0,"(seqc_0_loc, seqc_0_shape, syntax_slot_1, epoc...",NaN,0,230728
1,"(1, (-1, 1), line-6-2-0, 1, llCV2, 5)","(1, (-1, 1), line-6-2-0, 2, llCV2, 5)",1.536839,"(11, 15)",1.610711,2.772302,0.554355,0.010055,0.544300,1,1,1|1,True,"(-1, 1)","(-1, 1)","(-1, 1)|(-1, 1)",True,line-6-2-0,line-6-2-0,line-6-2-0|line-6-2-0,True,1,2,1|2,False,llCV2,llCV2,llCV2|llCV2,True,5,5,5|5,True,1|1|1|0|1|1,6,5.158629,0.339171,"((-1, 1), line-6-2-0, 1, llCV2, 5)","((-1, 1), line-6-2-0, 2, llCV2, 5)",False,M1,03_samp,"(epoch, FEAT_num_strokes_beh, seqc_0_loc, seqc...",0,syntax_slot_0,"(seqc_0_loc, seqc_0_shape, syntax_slot_1, epoc...",NaN,0,230728
2,"(1, (-1, 1), line-6-2-0, 1, llCV2, 5)","(1, (0, 0), line-6-2-0, 1, llCV2, 5)",1.639922,"(11, 9)",1.610711,2.772302,0.591538,0.056407,0.535131,1,1,1|1,True,"(-1, 1)","(0, 0)","(-1, 1)|(0, 0)",False,line-6-2-0,line-6-2-0,line-6-2-0|line-6-2-0,True,1,1,1|1,True,llCV2,llCV2,llCV2|llCV2,True,5,5,5|5,True,1|0|1|1|1|1,6,0.011681,0.291536,"((-1, 1), line-6-2-0, 1, llCV2, 5)","((0, 0), line-6-2-0, 1, llCV2, 5)",False,M1,03_samp,"(epoch, FEAT_num_strokes_beh, seqc_0_loc, seqc...",0,syntax_slot_0,"(seqc_0_loc, seqc_0_shape, syntax_slot_1, epoc...",NaN,0,230728
3,"(1, (-1, 1), line-6-2-0, 1, llCV2, 5)","(1, (0, 0), line-6-2-0, 2, llCV2, 5)",1.778989,"(11, 14)",1.610711,2.772302,0.641701,0.094700,0.547001,1,1,1|1,True,"(-1, 1)","(0, 0)","(-1, 1)|(0, 0)",False,line-6-2-0,line-6-2-0,line-6-2-0|line-6-2-0,True,1,2,1|2,False,llCV2,llCV2,llCV2|llCV2,True,5,5,5|5,True,1|0|1|0|1|1,6,0.119484,0.459285,"((-1, 1), line-6-2-0, 1, llCV2, 5)","((0, 0), line-6-2-0, 2, llCV2, 5)",False,M1,03_samp,"(epoch, FEAT_num_strokes_beh, seqc_0_loc, seqc...",0,syntax_slot_0,"(seqc_0_loc, seqc_0_shape, syntax_slot_1, epoc...",NaN,0,230728
4,"(1, (-1, 1), line-6-2-0, 1, llCV2, 5)","(1, (0, 0), line-6-2-0, 3, llCV2, 5)",1.689468,"(11, 8)",1.610711,2.772302,0.609410,0.058021,0.551389,1,1,1|1,True,"(-1, 1)","(0, 0)","(-1, 1)|(0, 0)",False,line-6-2-0,line-6-2-0,line-6-2-0|line-6-2-0,True,1,3,1|3,False,llCV2,llCV2,llCV2|llCV2,True,5,5,5|5,True,1|0|1|0|1|1,6,5.133569,0.303218,"((-1, 1), line-6-2-0, 1, llCV2, 5)","((0, 0), line-6-2-0, 3, llCV2, 5)",False,M1,03_samp,"(epoch, FEAT_num_strokes_beh, seqc_0_loc, seqc...",0,syntax_slot_0,"(seqc_0_loc, seqc_0_shape, syntax_slot_1, epoc...",NaN,0,230728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3000,"(3, (2, 0), line-6-2-0, 2, llCV2, 5)","(3, (1, 1), line-6-2-0, 1, llCV2, 5)",1.516768,"(6, 12)",1.618778,2.790980,0.543454,0.005695,0.537758,2,1,2|1,False,"(2, 0)","(1, 1)","(2, 0)|(1, 1)",False,line-6-2-0

In [ ]:
# ADD: compare angles of regression coefficients
##### Compare angles of regression coefficients across all conditions.
from neuralmonkey.scripts.analy_syntax_good_eucl_trial import ordinalregress_2_regr_coeff_pairs
savedir = f"{SAVEDIR}/pairwise_regr_coeffs"
os.makedirs(savedir, exist_ok=True)
ordinalregress_2_regr_coeff_pairs(DFWITHIN, savedir=savedir)

##### [DEbugging].. failure in agging...

In [ ]:
# NOTE: Solved -- now agging within each (var, vars_others) instead of after combining.

In [ ]:
path = "/tmp/df.pkl"
df = pd.read_pickle(path)
np.stack(df["coeff"])
from pythonlib.tools.pandastools import pad_tuple_values_to_same_length
for col in ["grp", "vars_grp"]:
    pad_tuple_values_to_same_length(df, col, col)

def F(x):
    X = np.stack(x)
    return np.mean(X, axis=0)
# group = ['grp', 'vars_grp', 'yvar']
group = ['grp']
# agg = {'coeff': F, 'balanced_accuracy': ['mean'], 'balanced_accuracy_adjusted': ['mean'], 'accuracy': ['mean'], 'score_train': ['mean'], 'n_labels_train': ['mean']}
# agg = {'balanced_accuracy': ['mean'], 'balanced_accuracy_adjusted': ['mean'], 'accuracy': ['mean'], 'score_train': ['mean'], 'n_labels_train': ['mean']}
agg = {'accuracy': ['mean']}
from pythonlib.tools.pandastools import stringify_values
# tmp = df[""]
df = stringify_values(df)
dfagg = df[1240:].groupby(group).agg(agg).reset_index()
df.loc[1230:1240, :]
df["grp"].value_counts()
from pythonlib.tools.pandastools import pad_tuple_values_to_same_length
pad_tuple_values_to_same_length(df, "grp", "grp")

##### [Debugging] Comparing old and new results for angle scores

In [ ]:
# NOTE: solved. Recent run was wrong becuase it was taking angle using just first 2 dims, even though the space is 6d. Code updated to fix this.

In [ ]:
dfangle_old = pd.read_pickle("/lemur2/lucas/analyses/recordings/main/syntax_good_trial/targeted_dim_redu_angles/Diego-230726-comb=False-q=RULE_BASE_trial/DFANGLE.pkl")
dfangle_new = pd.read_pickle("/lemur2/lucas/analyses/recordings/main/syntax_good_trial/targeted_pca_v2/Diego-230726-q=RULE_BASE_trial/03_samp-vlPFC/DFANGLE.pkl")
# dfangle_old = dfangle_old[dfangle_old["bregion"].isin(["preSMA_p", "preSMA_a"])].reset_index(drop=True)
dfangle_old = dfangle_old[dfangle_old["bregion"].isin(["vlPFC_p"])].reset_index(drop=True)

dfangle_old["n_levs"] = dfangle_old["levs_exist"].map(len)
dfangle_new["n_levs"] = dfangle_new["levs_exist"].map(len)
# dfangle_new = stringify_values(dfangle_new)
dfangle_new
dfangle_old
len(dfangle_old), len(dfangle_new)
dfangle_old[(dfangle_old["var_vector_length"]=="dist_yue_diff") & (dfangle_old["length_method"]=="dot") & (dfangle_old["n_levs"]==3)]

dfangle_new[(dfangle_new["var_vector_length"]=="dist_yue_diff") & (dfangle_new["length_method"]=="dot") & (dfangle_new["n_levs"]==3)]
